# Integrazione CoreML di RusTorch - Kernel Rust

Questo notebook dimostra come utilizzare CoreML con RusTorch.
Viene eseguito sul kernel Rust.

## Verificare Dipendenze e Funzionalità Richieste

In [ ]:
// Uso base di RusTorch
extern crate rustorch;

use rustorch::tensor::Tensor;
use rustorch::gpu::DeviceType;

println!("Versione RusTorch: {}", env!("CARGO_PKG_VERSION"));
println!("Versione Rust: {}", env!("RUSTC_VERSION"));

## Verificare Disponibilità CoreML

Verificare se CoreML è disponibile sul sistema corrente.

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::DeviceManager;
    
    let coreml_available = DeviceManager::is_coreml_available();
    println!("CoreML disponibile: {}", coreml_available);
    
    if coreml_available {
        println!("🎉 CoreML è disponibile!");
        println!("Piattaforma: macOS");
        
        // Mostrare informazioni dispositivo
        use rustorch::gpu::coreml::device_cache::DeviceCache;
        let cache = DeviceCache::global();
        cache.warmup();
        
        let stats = cache.get_stats();
        println!("Statistiche cache: {:?}", stats);
    } else {
        println!("⚠️ CoreML non è disponibile");
        println!("Si prega di usare CPU o altri backend GPU");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("❌ Le funzionalità CoreML non sono abilitate");
    println!("Si prega di compilare con --features coreml");
}

## Operazioni Tensore di Base

In [ ]:
// Creare tensori di base
let a = Tensor::zeros(&[2, 3]);
let b = Tensor::ones(&[3, 2]);

println!("Forma tensore A: {:?}", a.shape());
println!("Forma tensore B: {:?}", b.shape());

// Moltiplicazione matrici di base
let result = a.matmul(&b);
println!("Forma risultato: {:?}", result.shape());
println!("Operazioni tensore di base completate");

## Operazioni con Dispositivo CoreML

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::gpu::coreml::{CoreMLDevice, CoreMLBackend};
    use rustorch::backends::BackendConfig;
    
    // Tentare di creare dispositivo CoreML
    match CoreMLDevice::new(0) {
        Ok(device) => {
            println!("🖥️ Dispositivo CoreML creato con successo");
            println!("ID dispositivo: {}", device.id());
            println!("Disponibile: {}", device.is_available());
            println!("Limite memoria: {} MB", device.memory_limit() / (1024 * 1024));
            
            // Creare configurazione backend
            let config = BackendConfig::new()
                .with_caching(true)
                .with_max_cache_size(200)
                .with_profiling(true)
                .with_auto_fallback(true);
            
            println!("⚙️ Configurazione backend: {:?}", config);
            
            // Creare backend CoreML
            match CoreMLBackend::new(device, config) {
                Ok(backend) => {
                    println!("🚀 Backend CoreML inizializzato");
                    
                    // Ottenere statistiche
                    let stats = backend.get_statistics();
                    println!("📊 Statistiche backend:");
                    println!("  Operazioni totali: {}", stats.total_operations);
                    println!("  Hit cache: {}", stats.cache_hits);
                    println!("  Miss cache: {}", stats.cache_misses);
                    println!("  Operazioni fallback: {}", stats.fallback_operations);
                    
                    // Creare tensori su CoreML
                    let tensor_a = Tensor::randn(&[64, 64]).to_device(&backend);
                    let tensor_b = Tensor::randn(&[64, 64]).to_device(&backend);
                    
                    println!("📐 Tensori creati su dispositivo CoreML");
                    
                    // Operazione moltiplicazione matrici
                    let start = std::time::Instant::now();
                    let result = tensor_a.matmul(&tensor_b);
                    let duration = start.elapsed();
                    
                    println!("✅ Moltiplicazione matrici completata");
                    println!("⏱️ Tempo esecuzione: {:?}", duration);
                    println!("🎯 Forma risultato: {:?}", result.shape());
                    
                    // Pulire cache
                    backend.cleanup_cache();
                    println!("🧹 Cache pulita");
                }
                Err(e) => println!("❌ Errore creazione backend CoreML: {:?}", e),
            }
        }
        Err(e) => {
            println!("❌ Errore creazione dispositivo CoreML: {:?}", e);
            println!("CoreML potrebbe non essere disponibile su questo sistema");
        }
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Saltando operazioni CoreML - funzionalità non abilitate");
}

## Confronto Prestazioni

In [ ]:
use std::time::Instant;

fn benchmark_operations() {
    let sizes = vec![(64, 64), (128, 128), (256, 256), (512, 512)];
    
    println!("🏁 Benchmarking operazioni:");
    println!("Dimensione\t\tCPU (ms)\tDispositivo Preferito");
    println!("-" * 50);
    
    for (rows, cols) in sizes {
        // Creare tensori su CPU
        let a = Tensor::randn(&[rows, cols]);
        let b = Tensor::randn(&[cols, rows]);
        
        // Misurare tempo CPU
        let start = Instant::now();
        let _result = a.matmul(&b);
        let cpu_duration = start.elapsed();
        
        // Determinare dispositivo preferito
        let preferred_device = if rows * cols < 1000 {
            "CPU"
        } else if rows * cols < 10000 {
            "Metal GPU"
        } else {
            "CoreML"
        };
        
        println!("{}x{}\t\t{:.2}\t\t{}", 
                rows, cols, 
                cpu_duration.as_millis() as f64, 
                preferred_device);
    }
}

benchmark_operations();
println!("\n📝 Nota: La selezione dispositivo è basata su dimensione tensore e disponibilità");

## Selezione Intelligente Dispositivo

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::{DeviceManager, DeviceSelector};
    
    fn demonstrate_device_selection() {
        println!("🎯 Dimostrazione selezione intelligente dispositivo:");
        
        let operations = vec![
            ("Moltiplicazione piccola", vec![16, 16], "CPU"),
            ("Convoluzione 2D", vec![32, 3, 224, 224], "CoreML"),
            ("Trasformazione matrice", vec![128, 128], "Metal GPU"),
            ("Operazione batch grande", vec![512, 512], "CoreML"),
            ("Calcolo vettoriale", vec![1000], "CPU"),
        ];
        
        for (name, shape, preferred) in operations {
            println!("  {:<25} {:?} -> {}", name, shape, preferred);
            
            // Simulare selezione basata su regole
            let tensor_size: usize = shape.iter().product();
            let selected_device = match tensor_size {
                size if size < 1000 => DeviceType::Cpu,
                size if size < 50000 => DeviceType::MetalGpu,
                _ => {
                    if DeviceManager::is_coreml_available() {
                        DeviceType::CoreML
                    } else {
                        DeviceType::MetalGpu
                    }
                }
            };
            
            println!("    -> Dispositivo selezionato: {:?}", selected_device);
        }
        
        println!("\n📝 Logica selezione:");
        println!("  • < 1K elementi: CPU (overhead minimo)");
        println!("  • 1K-50K elementi: Metal GPU (bilanciato)");
        println!("  • > 50K elementi: CoreML (ottimizzato) o Metal GPU (fallback)");
    }
    
    demonstrate_device_selection();
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Dimostrazione selezione dispositivo saltata - funzionalità CoreML non disponibili");
}

## Esempio Avanzato: Strato Rete Neurale

In [ ]:
fn simulate_neural_layer() {
    println!("🧠 Simulazione strato rete neurale:");
    
    // Configurazione strato
    let batch_size = 32;
    let input_dim = 784;   // 28x28 MNIST
    let hidden_dim = 256;
    let output_dim = 10;   // 10 classi
    
    println!("📊 Configurazione:");
    println!("  Dimensione batch: {}", batch_size);
    println!("  Dimensione input: {}", input_dim);
    println!("  Dimensione nascosta: {}", hidden_dim);
    println!("  Dimensione output: {}", output_dim);
    
    // Creare tensori
    let input = Tensor::randn(&[batch_size, input_dim]);
    let weight1 = Tensor::randn(&[input_dim, hidden_dim]);
    let weight2 = Tensor::randn(&[hidden_dim, output_dim]);
    
    println!("\n🔄 Forward pass:");
    
    // Forward pass simulato
    let start = Instant::now();
    
    // Strato 1: input -> nascosto
    let hidden = input.matmul(&weight1);
    println!("  ✅ Input -> Nascosto: {:?}", hidden.shape());
    
    // Funzione attivazione ReLU (simulata)
    let activated = hidden.relu();
    println!("  ✅ Attivazione ReLU applicata");
    
    // Strato 2: nascosto -> output
    let output = activated.matmul(&weight2);
    println!("  ✅ Nascosto -> Output: {:?}", output.shape());
    
    let total_time = start.elapsed();
    
    println!("\n⏱️ Tempo totale forward pass: {:?}", total_time);
    println!("🚀 Prestazioni stimate: {:.0} campioni/secondo", 
             (batch_size as f64) / total_time.as_secs_f64());
    
    println!("\n📝 In un'implementazione reale:");
    println!("  • Matrici grandi utilizzerebbero CoreML");
    println!("  • Attivazioni utilizzerebbero Metal GPU");
    println!("  • Operazioni piccole rimarrebbero su CPU");
}

simulate_neural_layer();

## Gestione Errori e Fallback

In [ ]:
fn demonstrate_fallback_behavior() {
    println!("🔄 Dimostrazione comportamento fallback:");
    
    // Simulare operazione che potrebbe fallire su CoreML
    let complex_tensor = Tensor::randn(&[100, 100]);
    
    println!("🎯 Tentativo operazione CoreML...");
    
    // In implementazione reale, questo sarebbe:
    // match tensor.to_coreml() {
    //     Ok(coreml_tensor) => { /* usare CoreML */ },
    //     Err(_) => { /* fallback a Metal/CPU */ }
    // }
    
    let use_coreml = false; // Simulare fallimento CoreML
    
    if use_coreml {
        println!("✅ Operazione CoreML riuscita");
    } else {
        println!("⚠️ CoreML non disponibile, usando fallback");
        
        // Fallback a Metal GPU
        let start = Instant::now();
        let result = complex_tensor.matmul(&complex_tensor);
        let fallback_time = start.elapsed();
        
        println!("✅ Operazione fallback completata");
        println!("⏱️ Tempo fallback: {:?}", fallback_time);
        println!("📐 Forma risultato: {:?}", result.shape());
    }
    
    println!("\n📝 Strategia fallback:");
    println!("  1. Tentare CoreML (migliori prestazioni)");
    println!("  2. Fallback a Metal GPU (buona compatibilità)");
    println!("  3. Fallback finale a CPU (massima compatibilità)");
}

demonstrate_fallback_behavior();

## Riepilogo e Prossimi Passi

In [ ]:
println!("📋 Riepilogo Integrazione CoreML RusTorch (Kernel Rust):");
println!();
println!("✅ Funzionalità dimostrate:");
println!("  • Verifica disponibilità CoreML");
println!("  • Creazione e gestione dispositivi");
println!("  • Configurazione backend");
println!("  • Operazioni tensore di base");
println!("  • Benchmarking prestazioni");
println!("  • Selezione intelligente dispositivo");
println!("  • Comportamento fallback");
println!();
println!("🚧 Area sviluppo:");
println!("  • Implementazione completa operazioni CoreML");
println!("  • Ottimizzazione trasferimento memoria");
println!("  • Supporto esteso tipi tensore");
println!("  • Profiling prestazioni dettagliato");
println!("  • Integrazione con pipeline ML");
println!();
println!("🎯 Prossimi passi raccomandati:");
println!("  1. Testare con modelli CoreML preaddestrati");
println!("  2. Benchmark comparativo con altri backend");
println!("  3. Ottimizzare per casi d'uso specifici");
println!("  4. Deployare in applicazioni produzione");
println!();

#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    if rustorch::backends::DeviceManager::is_coreml_available() {
        println!("🎉 Tutte le funzionalità CoreML sono disponibili per test!");
    } else {
        println!("⚠️ CoreML è abilitato ma non disponibile su questo sistema");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Compilare con funzionalità CoreML per funzionalità completa");
}

println!("\n🚀 Pronto per sviluppo avanzato CoreML con RusTorch!");